<a href="https://colab.research.google.com/github/fernandopr11/ModelFaceInceptionV1FineTuning/blob/main/FacialRecognitionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalacion de dependencias
!pip install torch torchvision facenet-pytorch opencv-python pillow numpy pandas matplotlib seaborn scikit-learn tqdm albumentations mtcnn coremltools onnx onnx-coreml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 7.3 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=f73d771797b0209c6fd2fac656b84e3ed2897f7acd4bbfeb6d41cb8896e12331
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing


In [ ]:
#Importacion de librerias iniciales para la configuracion de directorios y carga del modelo preentrenado
import torch
import torch.nn as nn
from facenet_pytorch import InceptionResnetV1
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab import drive

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuración
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")


Mounted at /content/drive
Dispositivo: cpu


In [ ]:
# Crear directorios en Google Drive
base_path = Path("/content/drive/MyDrive/Reconocimiento_Facial")
models_dir = base_path / "models"
data_dir = base_path / "data"
results_dir = base_path / "results"

# Crear estructura de directorios
base_path.mkdir(exist_ok=True)
models_dir.mkdir(exist_ok=True)
data_dir.mkdir(exist_ok=True)
results_dir.mkdir(exist_ok=True)


In [ ]:
# Subdirectorios para organizar mejor los datos
(data_dir / "raw").mkdir(exist_ok=True)
(data_dir / "processed").mkdir(exist_ok=True)
(data_dir / "train_test").mkdir(exist_ok=True)

In [ ]:
# Veriricar la creacion de los directorios
print(f"Directorios creados en: {base_path}")
print(f"📁 models: {models_dir}")
print(f"📁 data: {data_dir}")
print(f"📁 results: {results_dir}")

Directorios creados en: /content/drive/MyDrive/Reconocimiento_Facial
📁 models: /content/drive/MyDrive/Reconocimiento_Facial/models
📁 data: /content/drive/MyDrive/Reconocimiento_Facial/data
📁 results: /content/drive/MyDrive/Reconocimiento_Facial/results


In [ ]:
print("Cargando InceptionResnetV1...")
model = InceptionResnetV1(
    pretrained='vggface2',  # Pesos preentrenados en VGGFace2
    classify=False,         # Solo extracción de características
    device=device
).eval()

Cargando InceptionResnetV1...


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
print(f"Modelo cargado exitosamente")
print(f"Parámetros: {sum(p.numel() for p in model.parameters()):,}")

Modelo cargado exitosamente
Parámetros: 27,910,327


In [ ]:
# Prueba de funcionamiento
test_input = torch.randn(1, 3, 160, 160).to(device)
with torch.no_grad():
    embedding = model(test_input)

print(f"Input shape: {test_input.shape}")
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding size: {embedding.shape[1]} (512 esperado)")

Input shape: torch.Size([1, 3, 160, 160])
Embedding shape: torch.Size([1, 512])
Embedding size: 512 (512 esperado)


In [ ]:
# Guardar el modelo en Google Drive
model_path = models_dir / "inception_resnet_v1_base.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'config': {
        'pretrained': 'vggface2',
        'embedding_size': 512,
        'img_size': 160
    }
}, model_path)

print(f"Modelo guardado en: {model_path}")

Modelo guardado en: /content/drive/MyDrive/Reconocimiento_Facial/models/inception_resnet_v1_base.pth
